In [10]:
# Импорт всех необходимых библиотек

import numpy as np
import torch
from torch.nn.functional import conv2d as libConv2d

In [11]:
class myfunc_conv2D():  # Определение класса для реализации свертки
    def __init__(  # Инициализация объекта класса со всеми необходимыми параметрами
        self, 
        input_data, 
        kernel_size: tuple | int, 
        bias: float | None = None, 
        stride: int = 1,  
        padding: tuple[int, int] | int | str = (0, 0),
        dilation: int = 1, 
    ):
        self.input_data = input_data[0, 0].numpy()  # Преобразование входных данных в numpy массив
        self.input_data_for_torch = input_data  
        self.bias = bias  

        if type(kernel_size) == tuple:  # Проверка типа ядра свертки
            self.kernel_size = kernel_size  
        else:
            self.kernel_size = (kernel_size, kernel_size)  # Преобразование в кортеж, если ядро - целое число

        self.stride = stride  
        self.dilation = dilation 

        if type(padding) == tuple:  # Проверка типа параметра паддинга
            self.padding = padding[0]  
        elif padding == "same":  # Если паддинг должен быть "same"
            if self.stride != 1:  # Проверка, что шаг свертки равен 1
                raise ValueError("padding 'same' works only with stride=1")  # Ошибка, если шаг не равен 1
            self.padding = self.kernel_size[0] - 1 
        elif padding == "valid": 
            self.padding = 0 
        else:
            self.padding = (padding, padding)  # Преобразование в кортеж, если параметр паддинга - число

        self.weight_tensor_for_torch = torch.randn(1, 1, self.kernel_size[0], self.kernel_size[1])  # Генерация весов для библиотеки torch
        self.weight_tensor = self.weight_tensor_for_torch[0, 0].numpy() 

    def conv2d(self):  # Реализация операции свертки
        image_height, image_width = self.input_data.shape  
        weight_height, weight_width = self.weight_tensor.shape 

        H_out = int((image_height - self.dilation * (weight_height - 1) - 1 + 2 * self.padding) / self.stride) + 1  # Вычисление высоты выхода
        W_out = int((image_width - self.dilation * (weight_width - 1) - 1 + 2 * self.padding) / self.stride) + 1  # Вычисление ширины выхода

        if self.padding > 0:
            self.input_data = np.pad(self.input_data, self.padding, mode='constant')  # Применение паддинга к входным данным

        result = np.zeros((H_out, W_out)) 

        for y in range(H_out):  # Цикл по высоте выхода
            for x in range(W_out):  # Цикл по ширине выхода
                input_slice = self.input_data[y * self.stride:y * self.stride + weight_height,
                               x * self.stride:x * self.stride + weight_width]  # Выделение среза входных данных
                result[y, x] = np.sum(input_slice * self.weight_tensor)  # Свертка и сохранение результата

        if self.bias: 
            result += self.bias  # Добавление смещения

        return result  

    def torch_conv2d(self):  # Вызов операции свертки из библиотеки torch
        return libConv2d(self.input_data_for_torch, self.weight_tensor_for_torch, self.bias, self.stride,
                         self.padding, self.dilation)
    
    # Проверка работы моей функции свертки по сравнению с библиотекой torch
    def test(self, print_flg=False):  
        my_conv2d = self.conv2d() 
        torch_out = np.array(self.torch_conv2d())  
        if print_flg:  
            print(my_conv2d)  
            print(torch_out[0, 0]) 
        print(np.allclose(my_conv2d, torch_out[0, 0]))  # Проверка совпадения результатов свертки


### Дальше идет проверка как раз-таки работоспособности моей функции сразу тест на сравнение со сверткой из торча, совпадают ли наши выходы и правильно ли все работает в общем:

#### ТЕСТ 1

In [18]:
image = torch.randn(1,1,5,5)
test1_out = myfunc_conv2D(image, kernel_size=1)
test1_out.test()

True


#### ТЕСТ 2

In [19]:
image = torch.randn(1,1,5,5)
test2_out = myfunc_conv2D(image, kernel_size=1, padding='valid')
test2_out.test()

True


#### ТЕСТ 3

In [20]:
image = torch.randn(1,1,5,5)
test3_out = myfunc_conv2D(image, kernel_size=1, padding='same')
test3_out.test()

True


#### ТЕСТ 4

In [21]:
image = torch.randn(1,1,5,5)
test4_out = myfunc_conv2D(image, kernel_size=4, padding='same')
test4_out.test()

True


#### ТЕСТ 5

In [22]:
image = torch.randn(1,1,5,5)
test5_out = myfunc_conv2D(image, kernel_size=1, dilation=3)
test5_out.test()

True


#### ТЕСТ 6

In [23]:
image = torch.randn(1,1,5,5)
test6_out = myfunc_conv2D(image, kernel_size=1, stride=4)
test6_out.test()

True


### Как видно из результатов выше -  функция свертки работает отлично, результаты с оригинальной из торча совпадают с моей!